In [1]:
import time
import numpy as np  
import cupy as cp

#Componentes de keras con tensorflow

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras import optimizers, backend
import tensorflow.keras.utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.datasets import fashion_mnist #Descargar la base de datos

In [2]:
#--------------------Descarga de DataSet---------------------------------------

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

#---------------------Preprocesamiento de Datos--------------------------------
# Aplanamiento
# x_train.reshape(cantidad_de_imágenes, 784)
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

# Escalamiento
# Escala los valores de los pixeles al rango 0 -> 1
x_train = x_train/255
x_test = x_test/255

#x_train_gpu=cp.asarray(x_train)
#x_train_gpu=cp.divide(x_train_gpu, 255.0)
#x_train = cp.asnumpy(x_train_gpu)

#x_test_gpu=cp.asarray(x_test)
#x_test_gpu=cp.divide(x_test_gpu, 255.0)
#x_test = cp.asnumpy(x_test_gpu)

# Codificación 1-hot
# Aplica codificación 1-hot a las etiquetas y almacena las nuevas etiquetas en variables alternativas y_train1 y y_test1
y_train1 = np_utils.to_categorical(y_train)
y_test1 = np_utils.to_categorical(y_test)

In [3]:
#--------------------Creacion del Modelo---------------------------------------

# Escribe la instrucción que elimina los modelos creados con keras 
# Recuerda que es necesario eliminar el modelo cada que se desee volver a entrenar
backend.clear_session()

In [4]:
modeloConv = Sequential()
modeloConv.add(Conv2D(64, kernel_size=(2,2), activation='relu',
                      input_shape = (28,28,1)))
modeloConv.add(MaxPooling2D(pool_size=(2,2)))
modeloConv.add(Conv2D(128, kernel_size=(2,2), activation='relu'))
modeloConv.add(MaxPooling2D(pool_size=(2,2)))
modeloConv.add(Flatten())
modeloConv.add(Dense(30, activation='relu'))
modeloConv.add(Dense(20, activation='relu'))
modeloConv.add(Dense(10, activation='softmax'))
# Fin de la definición del modelo

# La siguiente línea de código es para mostrar la tabla con las características del modelo
modeloConv.summary() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 27, 27, 64)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 12, 12, 128)       32896     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 128)        0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 30)                1

In [5]:
# Define el optimizador Adam
adam = optimizers.Adam(learning_rate = 0.001)

# Compila el modelo definiendo el optimizador "adam", la función de costo (loss) como la "categorical_crossentropy"
# y la metrica como el "accuracy"
modeloConv.compile(optimizer=adam, 
                 loss='categorical_crossentropy', 
                 metrics=['accuracy'])

In [6]:
# Entrena el modelo eligiendo un número de epocas y los datos de entrenamiento y validación
# Recuerda que guardamos el entrenamiento en una variable para poder graficar el error
# RECUERDA ENTRENAR CON LOS "Y" CODIFICADOS CON 1-HOT

inicio_time = time.perf_counter()

m = modeloConv.fit(x_train, y_train1, 
                 validation_data = (x_test, y_test1),
                 epochs = 20, batch_size = 70,
                 verbose = True)

fin_time = time.perf_counter()

print('Tiempo de ejecución = ',fin_time-inicio_time)

Epoch 1/20
858/858 [==============================] - 7s 6ms/step - loss: 0.5326 - accuracy: 0.8093 - val_loss: 0.3924 - val_accuracy: 0.8597
Epoch 2/20
858/858 [==============================] - 5s 5ms/step - loss: 0.3421 - accuracy: 0.8761 - val_loss: 0.3481 - val_accuracy: 0.8741
Epoch 3/20
858/858 [==============================] - 5s 5ms/step - loss: 0.2997 - accuracy: 0.8903 - val_loss: 0.3022 - val_accuracy: 0.8904
Epoch 4/20
858/858 [==============================] - 5s 6ms/step - loss: 0.2704 - accuracy: 0.9018 - val_loss: 0.2986 - val_accuracy: 0.8917
Epoch 5/20
858/858 [==============================] - 5s 6ms/step - loss: 0.2498 - accuracy: 0.9080 - val_loss: 0.2808 - val_accuracy: 0.8994
Epoch 6/20
858/858 [==============================] - 5s 6ms/step - loss: 0.2290 - accuracy: 0.9164 - val_loss: 0.2800 - val_accuracy: 0.9020
Epoch 7/20
858/858 [==============================] - 5s 6ms/step - loss: 0.2141 - accuracy: 0.9217 - val_loss: 0.2714 - val_accuracy: 0.9022
Epoch 

In [7]:
#----------------------División del dataset-----------------------------------

x_train_0 = x_train[:20000]
x_train_1 = x_train[20001:40000]
x_train_2 = x_train[40000:]

y_train1_0 = y_train1[:20000]
y_train1_1 = y_train1[20001:40000]
y_train1_2 = y_train1[40000:]

x_test_0 = x_test[:3333]
x_test_1 = x_test[3334:6666]
x_test_2 = x_test[6667:]

y_test1_0 = y_test1[:3333]
y_test1_1 = y_test1[3334:6666]
y_test1_2 = y_test1[6667:]

In [8]:
#-------------------Creación copia 0 del modelo------------------------------

backend.clear_session()

modeloConv0  = keras.models.clone_model(modeloConv)

#modeloConv0.summary()

modeloConv0.compile(optimizer=adam, 
                 loss='categorical_crossentropy', 
                 metrics=['accuracy'])

In [9]:
inicio_time = time.perf_counter()

m0 = modeloConv0.fit(x_train_0, y_train1_0, 
                 validation_data = (x_test_0, y_test1_0),
                 epochs = 20, batch_size = 70,
                 verbose = True)

fin_time = time.perf_counter()

print('Tiempo de ejecución = ',fin_time-inicio_time)

Epoch 1/20
286/286 [==============================] - 2s 6ms/step - loss: 0.6261 - accuracy: 0.7647 - val_loss: 0.4395 - val_accuracy: 0.8464
Epoch 2/20
286/286 [==============================] - 2s 5ms/step - loss: 0.3769 - accuracy: 0.8626 - val_loss: 0.3831 - val_accuracy: 0.8641
Epoch 3/20
286/286 [==============================] - 2s 5ms/step - loss: 0.3283 - accuracy: 0.8778 - val_loss: 0.3769 - val_accuracy: 0.8659
Epoch 4/20
286/286 [==============================] - 2s 5ms/step - loss: 0.3002 - accuracy: 0.8892 - val_loss: 0.3654 - val_accuracy: 0.8710
Epoch 5/20
286/286 [==============================] - 2s 5ms/step - loss: 0.2765 - accuracy: 0.8978 - val_loss: 0.3721 - val_accuracy: 0.8707
Epoch 6/20
286/286 [==============================] - 2s 5ms/step - loss: 0.2584 - accuracy: 0.9031 - val_loss: 0.3484 - val_accuracy: 0.8758
Epoch 7/20
286/286 [==============================] - 2s 5ms/step - loss: 0.2453 - accuracy: 0.9110 - val_loss: 0.3346 - val_accuracy: 0.8860
Epoch 

In [10]:
#-------------------Creación copia 1 del modelo------------------------------

backend.clear_session()

modeloConv1  = keras.models.clone_model(modeloConv)

#modeloConv1.summary()

modeloConv1.compile(optimizer=adam, 
                 loss='categorical_crossentropy', 
                 metrics=['accuracy'])

In [11]:
inicio_time = time.perf_counter()

m0 = modeloConv1.fit(x_train_1, y_train1_1, 
                 validation_data = (x_test_1, y_test1_1),
                 epochs = 20, batch_size = 70,
                 verbose = True)

fin_time = time.perf_counter()

print('Tiempo de ejecución = ',fin_time-inicio_time)

Epoch 1/20
286/286 [==============================] - 3s 7ms/step - loss: 0.6064 - accuracy: 0.7804 - val_loss: 0.4742 - val_accuracy: 0.8322
Epoch 2/20
286/286 [==============================] - 2s 5ms/step - loss: 0.3810 - accuracy: 0.8638 - val_loss: 0.4053 - val_accuracy: 0.8592
Epoch 3/20
286/286 [==============================] - 2s 5ms/step - loss: 0.3393 - accuracy: 0.8777 - val_loss: 0.3940 - val_accuracy: 0.8649
Epoch 4/20
286/286 [==============================] - 2s 5ms/step - loss: 0.3106 - accuracy: 0.8874 - val_loss: 0.3699 - val_accuracy: 0.8694
Epoch 5/20
286/286 [==============================] - 2s 5ms/step - loss: 0.2868 - accuracy: 0.8970 - val_loss: 0.3588 - val_accuracy: 0.8718
Epoch 6/20
286/286 [==============================] - 1s 5ms/step - loss: 0.2706 - accuracy: 0.9007 - val_loss: 0.3573 - val_accuracy: 0.8773
Epoch 7/20
286/286 [==============================] - 2s 5ms/step - loss: 0.2640 - accuracy: 0.9042 - val_loss: 0.3495 - val_accuracy: 0.8785
Epoch 

In [12]:
#-------------------Creación copia 2 del modelo------------------------------

backend.clear_session()

modeloConv2  = keras.models.clone_model(modeloConv)

#modeloConv2.summary()

modeloConv2.compile(optimizer=adam, 
                 loss='categorical_crossentropy', 
                 metrics=['accuracy'])

In [13]:
inicio_time = time.perf_counter()

m0 = modeloConv2.fit(x_train_2, y_train1_2, 
                 validation_data = (x_test_2, y_test1_2),
                 epochs = 20, batch_size = 70,
                 verbose = True)

fin_time = time.perf_counter()

print('Tiempo de ejecución = ',fin_time-inicio_time)

Epoch 1/20
286/286 [==============================] - 2s 6ms/step - loss: 0.6421 - accuracy: 0.7694 - val_loss: 0.4573 - val_accuracy: 0.8335
Epoch 2/20
286/286 [==============================] - 2s 5ms/step - loss: 0.3900 - accuracy: 0.8547 - val_loss: 0.3699 - val_accuracy: 0.8623
Epoch 3/20
286/286 [==============================] - 2s 5ms/step - loss: 0.3394 - accuracy: 0.8767 - val_loss: 0.3573 - val_accuracy: 0.8584
Epoch 4/20
286/286 [==============================] - 2s 5ms/step - loss: 0.3097 - accuracy: 0.8882 - val_loss: 0.3199 - val_accuracy: 0.8758
Epoch 5/20
286/286 [==============================] - 2s 5ms/step - loss: 0.2891 - accuracy: 0.8953 - val_loss: 0.3160 - val_accuracy: 0.8773
Epoch 6/20
286/286 [==============================] - 2s 5ms/step - loss: 0.2679 - accuracy: 0.9015 - val_loss: 0.3144 - val_accuracy: 0.8740
Epoch 7/20
286/286 [==============================] - 2s 5ms/step - loss: 0.2477 - accuracy: 0.9097 - val_loss: 0.3054 - val_accuracy: 0.8851
Epoch 